In [ ]:
#default_exp utils.engine

In [ ]:
#export
import os
import time
import timeit
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from abc import ABC
from fastprogress.fastprogress import master_bar, progress_bar

In [ ]:
#export
class Fitter(ABC):
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def log(self):
        pass
    
    def train(self):
        pass
    
    def validate(self):
        pass

In [ ]:
#export
class DialoGPTFitter(Fitter):
    def __init__(self, model, dataloaders, optimizer, device, log_file='training_log.txt',scheduler=None):
        self.model = model
        self.train_dl, self.valid_dl = dataloaders[0], dataloaders[1]
        self.optimizer = optimizer
        self.scheduler = scheduler
        if not os.path.exists(os.path.join('..', 'outputs')): os.makedirs(os.path.join('..', 'outputs'))
        if os.path.exists(os.path.join('..', 'outputs', f'{log_file}')): 
            os.remove(os.path.join('..', 'outputs', f'{log_file}'))
        self.log_file = os.path.join('..', 'outputs', f'{log_file}')
        self.device = device
        
    def fit(self, epochs, return_metric=False, 
            monitor='epoch train_loss valid_loss time', 
            model_path=os.path.join('..', 'weights', 'model.pth'), show_graph=True):
        self.model_path = model_path
        self.log(f'{time.ctime()}')
        self.log(f'Using device: {self.device}')
        mb = master_bar(range(1, epochs+1)) #MAJOR
        mb.write(monitor.split(),table=True)
        
        model = self.model.to(self.device)
        optimizer = self.optimizer
        scheduler = self.scheduler
        best_metric = -np.inf
        train_loss_list, valid_loss_list = [], []
        
        for i_, epoch in enumerate(mb):
            epoch_start = timeit.default_timer()
            start = time.time()
            self.log('-'*50)
            self.log(f'Running Epoch #{epoch} {"🔥"*epoch}')
            self.log(f'{"-"*50} \n')
            
            self.log('TRAINING...')
            train_loss = self.train(mb, model, optimizer, self.device, scheduler)
            train_loss_list.append(train_loss) #for graph
            self.log(f'Training time: {round(time.time()-start, 2)} secs \n')
            
            start = time.time()
            self.log('EVALUATING...')
            valid_loss = self.validate(mb, model, self.device)    
            valid_loss_list.append(valid_loss) #for graph
            
            if show_graph:
                self.plot_loss_update(epoch, epochs, mb, train_loss_list, valid_loss_list) # for graph
                               
            epoch_end = timeit.default_timer()
            total_time = epoch_end - epoch_start
            mins, secs = divmod(total_time, 60)
            hours, mins = divmod(mins, 60)
            ret_time = f'{int(hours)}:{int(mins)}:{int(secs)}'
            mb.write([epoch,f'{train_loss:.6f}',f'{valid_loss:.6f}',
                      f'{ret_time}'],table=True)
            self.log(f'Evaluation time: {ret_time}\n')
#             break
            
        if return_metric: return best_metric
    
    def train(self, mb, model, opt, device, sched=None):     
        model.train()
        train_loss = 0       
        for ind, xy in enumerate(progress_bar(self.train_dl, parent=mb)):
            xy = {key: xy_.to(device) for key, xy_ in xy.items()}
            opt.zero_grad()
            loss, *out = model(**xy)
            loss.backward()
            opt.step()       
            if sched is not None:
                sched.step()
            train_loss += loss.item()
            
            if ind % 500 == 0:
                self.log(f'Batch: {ind}, Train loss: {train_loss/ len(self.train_dl)}')
            
            mb.child.comment = f'{train_loss / (ind+1) :.3f}'
        return train_loss / len(self.train_dl)
    
    def validate(self, mb, model, device):    
        model.eval()
        valid_loss = 0      
        with torch.no_grad():
            for ind, xy in enumerate(progress_bar(self.valid_dl, parent=mb)):
                xy = {key: xy_.to(device) for key, xy_ in xy.items()}
                opt.zero_grad()
                loss, *out = model(**xy)
                
                valid_loss += loss.item()

                if ind % 500 == 0:
                    self.log(f'Batch: {ind}, Valid loss: {valid_loss / (ind+1) :.3f}')

                mb.child.comment = f'{valid_loss / (ind+1) :.3f}'  
        return valid_loss / len(self.valid_dl)
        
            
    def log(self, message, verbose=False):
        if verbose: print(message)
        with open(self.log_file, 'a+') as logger_:
            logger_.write(f'{message}\n')
            
    
    @staticmethod
    def plot_loss_update(epoch, epochs, mb, train_loss, valid_loss):
        """ dynamically print the loss plot during the training/validation loop.
            expects epoch to start from 1.
        """
        x = range(1, epoch+1)
        y = np.concatenate((train_loss, valid_loss))
        graphs = [[x,train_loss], [x,valid_loss]]
        x_margin = 0.2
        y_margin = 0.05
        x_bounds = [1-x_margin, epochs+x_margin]
        y_bounds = [np.min(y)-y_margin, np.max(y)+y_margin]

        mb.update_graph(np.array(graphs), np.array(x_bounds), np.array(y_bounds))